# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark
findspark.init('/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('kmeans').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/20 13:27:37 WARN Utils: Your hostname, Lanz-Lenovo, resolves to a loopback address: 127.0.1.1; using 192.168.1.145 instead (on interface wlp2s0)
25/08/20 13:27:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/20 13:27:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/20 13:27:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/20 13:27:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
dt = spark.read.csv('hack_data.csv', header=True, inferSchema=True)

In [4]:
dt.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [5]:
dt.count()

334

In [8]:
dt.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [6]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
                                         'Bytes Transferred',
                                         'Kali_Trace_Used',
                                         'Servers_Corrupted',
                                         'Pages_Corrupted',
                                         'WPM_Typing_Speed'],
                           outputCol='features')

In [13]:
data_assembled = assembler.transform(dt)

In [14]:
from pyspark.ml.feature import StandardScaler

In [15]:
scale = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [16]:
scaler_model = scale.fit(data_assembled)
final_data = scaler_model.transform(data_assembled)

In [17]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)



In [18]:
from pyspark.ml.clustering import KMeans

In [21]:
kmeans_2 = KMeans(k=2, seed=1, featuresCol='scaledFeatures')
kmeans_3 = KMeans(k=3, seed=1, featuresCol='scaledFeatures')

In [22]:
model_2 = kmeans_2.fit(final_data)
model_3 = kmeans_3.fit(final_data)

In [23]:
prediction_2 = model_2.transform(final_data)
prediction_3 = model_3.transform(final_data)

In [29]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [30]:
eva = ClusteringEvaluator(featuresCol='scaledFeatures')

In [31]:
print('With 2 hackers')
eva.evaluate(prediction_2)

With 2 hackers


0.8176460094012482

In [1]:
print('With 3 hackers')
eva.evaluate(prediction_3)

With 3 hackers


NameError: name 'eva' is not defined

## Review count of predictions

In [38]:
print('With 2 hackers')
prediction_2.groupBy('prediction').count().show()

With 2 hackers
+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [37]:
print('With 3 hackers')
prediction_3.groupBy('prediction').count().show()

With 3 hackers
+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   79|
|         0|   88|
+----------+-----+



Los resultados obtenidos del ClusteringEvaluator, con una puntuación de 0.81 de precisión, y la información proporcionada por el ingeniero, sugieren que la división de ataques entre los hackers es equitativa. Por lo tanto, se ha determinado que los ataques provienen de dos individuos en lugar de los tres que se habían considerado inicialmente.

The results from the ClusteringEvaluator, with a precision score of 0.81, combined with the information from the engineer indicating that the hackers distributed the attacks among themselves, lead to the conclusion that the attacks originate from two individuals rather than the initially assumed three.